In [15]:
import os
from bs4 import BeautifulSoup
import csv
import requests
import pandas


In [16]:
food_data=open("food.html","r",encoding="utf8")
food=food_data.read()
len(food)

3474604

In [17]:
soup=BeautifulSoup(food,"html.parser")


In [21]:
recipes=soup.find_all('div',{'class':"fd-tile fd-recipe"})
recipes=recipes[:880]
len(recipes)

880

In [22]:
recipe_list=[]
for recipe in recipes:
    title=recipe.find('h2',{'class':"title"}).text
    links=recipe.find("a",{'href':True})['href']
    recipe_list.append({'name':title,'link':links})
len(recipe_list)    

880

In [23]:
with open("recipeLinks.csv",'w',newline='') as csvfile:
    fieldnames=['name','link']
    writer=csv.DictWriter(csvfile,fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(recipe_list)
    
    